# Data Processing

In this notebook the following tasks will be done:
- Exploratory analisis
- Question statement
- Data processing based on exploratory analisis and target questions 

In [1]:
import polars as pl
import numpy as np
from PyPDF2 import PdfReader

In [2]:
raw_df: pl.DataFrame = pl.read_csv(
    "../data/raw/world-data-2023.csv", ignore_errors=True
)

# Fix some column names
raw_df = raw_df.rename({"Density\r\n(P/Km2)": "Density(P/Km2)"})
raw_df = raw_df.rename({"Agricultural Land( %)": "Agricultural Land(%)"})
raw_df = raw_df.rename({"Urban_population": "Urban population"})

# Remove rows that are completely empty
raw_df = raw_df.filter(
    ~pl.col("Country").is_in(["Palestinian National Authority", "Vatican City"])
)


pl.DataFrame(dict(zip(raw_df.columns, raw_df.dtypes)))

Country,Density(P/Km2),Abbreviation,Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,CPI,CPI Change (%),Currency-Code,Fertility Rate,Forested Area (%),Gasoline Price,GDP,Gross primary education enrollment (%),Gross tertiary education enrollment (%),Infant mortality,Largest city,Life expectancy,Maternal mortality ratio,Minimum wage,Official language,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban population,Latitude,Longitude
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
String,String,String,String,String,String,Float64,Int64,String,String,Float64,String,String,Float64,String,String,String,String,String,Float64,String,Float64,Int64,String,String,String,Float64,String,String,String,String,String,String,Float64,Float64


Visualize the original dataset

In [3]:
subset_df = raw_df.slice(150, 5)
subset_df

Country,Density(P/Km2),Abbreviation,Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,CPI,CPI Change (%),Currency-Code,Fertility Rate,Forested Area (%),Gasoline Price,GDP,Gross primary education enrollment (%),Gross tertiary education enrollment (%),Infant mortality,Largest city,Life expectancy,Maternal mortality ratio,Minimum wage,Official language,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban population,Latitude,Longitude
str,str,str,str,str,str,f64,i64,str,str,f64,str,str,f64,str,str,str,str,str,f64,str,f64,i64,str,str,str,f64,str,str,str,str,str,str,f64,f64
"""Senegal""","""87""","""SN""","""46.10%""","""196,722""","""19,000""",34.52,221,"""Dakar""","""10,902""",109.25,"""1.80%""","""XOF""",4.63,"""42.80%""","""$1.14 ""","""$23,578,084,052 ""","""81.00%""","""12.80%""",31.8,"""Pikine""",67.7,315,"""$0.31 ""","""French""","""44.20%""",0.07,"""16,296,364""","""45.70%""","""16.30%""","""44.80%""","""6.60%""","""7,765,706""",14.497401,-14.452362
"""Serbia""","""100""","""RS""","""39.30%""","""77,474""","""32,000""",9.2,381,"""Belgrade""","""45,221""",144.0,"""1.80%""","""RSD""",1.49,"""31.10%""","""$1.16 ""","""$51,409,167,351 ""","""100.30%""","""67.20%""",4.8,"""Belgrade""",75.5,12,"""$1.57 ""","""Serbian""","""40.60%""",3.11,"""6,944,975""","""54.90%""","""18.60%""","""36.60%""","""12.69%""","""3,907,243""",44.016521,21.005859
"""Seychelles""","""214""","""SC""","""3.40%""","""455""","""0""",17.1,248,"""Victoria, Seychelles""","""605""",129.96,"""1.80%""","""SCR""",2.41,"""88.40%""",null,"""$1,698,843,063 ""","""100.40%""","""17.10%""",12.4,"""Victoria, Seychelles""",72.8,53,"""$2.00 ""","""French""","""2.50%""",0.95,"""97,625""",null,"""34.10%""","""30.10%""",null,"""55,762""",-4.679574,55.491977
"""Sierra Leone""","""111""","""SL""","""54.70%""","""71,740""","""9,000""",33.41,232,"""Freetown""","""1,093""",234.16,"""14.80%""","""SLL""",4.26,"""43.10%""","""$1.08 ""","""$3,941,474,311 ""","""112.80%""","""2.00%""",78.5,"""Freetown""",54.3,1120,"""$0.57 ""","""English""","""38.20%""",0.03,"""7,813,215""","""57.90%""","""8.60%""","""30.70%""","""4.43%""","""3,319,366""",8.460555,-11.779889
"""Singapore""","""8,358""","""SG""","""0.90%""","""716""","""81,000""",8.8,65,null,"""37,535""",114.41,"""0.60%""","""SGD""",1.14,"""23.10%""","""$1.25 ""","""$372,062,527,489 ""","""100.60%""","""84.80%""",2.3,null,83.1,8,null,"""Malay""","""36.70%""",2.29,"""5,703,569""","""70.50%""","""13.10%""","""21.00%""","""4.11%""","""5,703,569""",1.352083,103.819836


In [4]:
subset_df2 = raw_df.slice(81, 5)
subset_df2

Country,Density(P/Km2),Abbreviation,Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Calling Code,Capital/Major City,Co2-Emissions,CPI,CPI Change (%),Currency-Code,Fertility Rate,Forested Area (%),Gasoline Price,GDP,Gross primary education enrollment (%),Gross tertiary education enrollment (%),Infant mortality,Largest city,Life expectancy,Maternal mortality ratio,Minimum wage,Official language,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban population,Latitude,Longitude
str,str,str,str,str,str,f64,i64,str,str,f64,str,str,f64,str,str,str,str,str,f64,str,f64,i64,str,str,str,f64,str,str,str,str,str,str,f64,f64
"""Israel""","""400""","""IL""","""24.60%""","""20,770""","""178,000""",20.8,972,"""Jerusalem""","""65,166""",108.15,"""0.80%""","""ILS""",3.09,"""7.70%""","""$1.57 ""","""$395,098,666,122 ""","""104.90%""","""63.40%""",3.0,"""Jerusalem""",82.8,3,"""$7.58 ""","""Hebrew""","""24.40%""",4.62,"""9,053,300""","""64.00%""","""23.10%""","""25.30%""","""3.86%""","""8,374,393""",31.046051,34.851612
"""Italy""","""206""","""IT""","""43.20%""","""301,340""","""347,000""",7.3,39,"""Rome""","""320,411""",110.62,"""0.60%""","""EUR""",1.29,"""31.80%""","""$1.61 ""","""$2,001,244,392,042 ""","""101.90%""","""61.90%""",2.6,"""Rome""",82.9,2,null,"""Italian""","""22.80%""",3.98,"""60,297,396""","""49.60%""","""24.30%""","""59.10%""","""9.89%""","""42,651,966""",41.87194,12.56738
"""Jamaica""","""273""","""JM""","""41.00%""","""10,991""","""4,000""",16.1,1876,"""Kingston, Jamaica""","""8,225""",162.47,"""3.90%""","""JMD""",1.98,"""30.90%""","""$1.11 ""","""$16,458,071,068 ""","""91.00%""","""27.10%""",12.4,"""Kingston, Jamaica""",74.4,80,"""$1.33 ""","""Jamaican English""","""23.70%""",1.31,"""2,948,279""","""66.00%""","""26.80%""","""35.10%""","""8.00%""","""1,650,594""",18.109581,-77.297508
"""Japan""","""347""","""JP""","""12.30%""","""377,944""","""261,000""",7.4,81,"""Tokyo""","""1,135,886""",105.48,"""0.50%""",null,1.42,"""68.50%""","""$1.06 ""","""$5,081,769,542,380 ""","""98.80%""","""63.20%""",1.8,"""Tokyo""",84.2,5,"""$6.77 ""","""None""","""13.10%""",2.41,"""126,226,568""","""61.70%""","""11.90%""","""46.70%""","""2.29%""","""115,782,416""",36.204824,138.252924
"""Jordan""","""115""","""JO""","""12.00%""","""89,342""","""116,000""",21.98,962,"""Amman""","""25,108""",125.6,"""0.80%""","""JOD""",2.76,"""1.10%""","""$1.10 ""","""$43,743,661,972 ""","""81.50%""","""34.40%""",13.9,"""Amman""",74.4,46,"""$1.49 ""","""Arabic""","""25.10%""",2.32,"""10,101,694""","""39.30%""","""15.10%""","""28.60%""","""14.72%""","""9,213,048""",30.585164,36.238414


Firstly, we are going to eliminate the columns that we consider unnecessary, given the notebook exploratory_analysis.ipynb

In [5]:
columns_to_drop = [
    "Calling Code",
    "Capital/Major City",
    "Largest City",
    "Official language",
    "Minimum wage currency",
    "CPI change",
]
columns_to_keep = [col for col in raw_df.columns if col not in columns_to_drop]
cleaned_df = raw_df.select(columns_to_keep)

Additionally, we must convert the columns to their corresponding type

In [6]:
# Columns that should be integers
int_columns = [
    "Density(P/Km2)",
    "Land Area(Km2)",
    "Armed Forces size",
    "Co2-Emissions",
    "Population",
    "Urban population",
]
cleaned_df = cleaned_df.with_columns(
    [
        pl.col(col).str.strip_chars().str.replace_all(",", "").cast(pl.Int64)
        for col in int_columns
    ]
)

# Currency columns that should be floats
float_columns_currency = ["Gasoline Price", "GDP", "Minimum wage"]
cleaned_df = cleaned_df.with_columns(
    [
        pl.col(col).str.strip_chars().str.replace_all(r"[\$,]", "").cast(pl.Float64)
        for col in float_columns_currency
    ]
)

# Percentage columns that should be floats
float_columns_percentage = [
    "Agricultural Land(%)",
    "CPI Change (%)",
    "Forested Area (%)",
    "Gross primary education enrollment (%)",
    "Gross tertiary education enrollment (%)",
    "Out of pocket health expenditure",
    "Population: Labor force participation (%)",
    "Tax revenue (%)",
    "Total tax rate",
    "Unemployment rate",
]
cleaned_df = cleaned_df.with_columns(
    [
        pl.col(col).str.strip_chars().str.replace_all("%", "").cast(pl.Float64)
        for col in float_columns_percentage
    ]
)

cleaned_df.head(10)

Country,Density(P/Km2),Abbreviation,Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Co2-Emissions,CPI,CPI Change (%),Currency-Code,Fertility Rate,Forested Area (%),Gasoline Price,GDP,Gross primary education enrollment (%),Gross tertiary education enrollment (%),Infant mortality,Largest city,Life expectancy,Maternal mortality ratio,Minimum wage,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban population,Latitude,Longitude
str,i64,str,f64,i64,i64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,i64,f64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64
"""Afghanistan""",60,"""AF""",58.1,652230,323000,32.49,8672,149.9,2.3,"""AFN""",4.47,2.1,0.7,1.9101e10,104.0,9.7,47.9,"""Kabul""",64.5,638,0.43,78.4,0.28,38041754,48.9,9.3,71.4,11.12,9797273,33.93911,67.709953
"""Albania""",105,"""AL""",43.1,28748,9000,11.78,4536,119.05,1.4,"""ALL""",1.62,28.1,1.36,1.5278e10,107.0,55.0,7.8,"""Tirana""",78.5,15,1.12,56.9,1.2,2854191,55.7,18.6,36.6,12.33,1747593,41.153332,20.168331
"""Algeria""",18,"""DZ""",17.4,2381741,317000,24.28,150006,151.36,2.0,"""DZD""",3.02,0.8,0.28,1.6999e11,109.9,51.4,20.1,"""Algiers""",76.7,112,0.95,28.1,1.72,43053054,41.2,37.2,66.1,11.7,31510100,28.033886,1.659626
"""Andorra""",164,"""AD""",40.0,468,null,7.2,469,null,null,"""EUR""",1.27,34.0,1.51,3.1541e9,106.4,null,2.7,"""Andorra la Vella""",null,null,6.63,36.4,3.33,77142,null,null,null,null,67873,42.506285,1.521801
"""Angola""",26,"""AO""",47.5,1246700,117000,40.73,34693,261.73,17.1,"""AOA""",5.52,46.3,0.97,9.4635e10,113.5,9.3,51.6,"""Luanda""",60.8,241,0.71,33.4,0.21,31825295,77.5,9.2,49.1,6.89,21061025,-11.202692,17.873887
"""Antigua and Barbuda""",223,"""AG""",20.5,443,0,15.33,557,113.81,1.2,"""XCD""",1.99,22.3,0.99,1.7278e9,105.0,24.8,5.0,"""St. John's, Saint John""",76.9,42,3.04,24.3,2.76,97118,null,16.5,43.0,null,23800,17.060816,-61.796428
"""Argentina""",17,"""AR""",54.3,2780400,105000,17.02,201348,232.75,53.5,"""ARS""",2.26,9.8,1.1,4.4966e11,109.7,90.0,8.8,"""Buenos Aires""",76.5,39,3.35,17.6,3.96,44938712,61.3,10.1,106.3,9.79,41339571,-38.416097,-63.616672
"""Armenia""",104,"""AM""",58.9,29743,49000,13.99,5156,129.18,1.4,"""AMD""",1.76,11.7,0.77,1.3673e10,92.7,54.6,11.0,"""Yerevan""",74.9,26,0.66,81.6,4.4,2957731,55.6,20.9,22.6,16.99,1869848,40.069099,45.038189
"""Australia""",3,"""AU""",48.2,7741220,58000,12.6,375908,119.8,1.6,"""AUD""",1.74,16.3,0.93,1.3927e12,100.3,113.1,3.1,"""Sydney""",82.7,6,13.59,19.6,3.68,25766605,65.5,23.0,47.4,5.27,21844756,-25.274398,133.775136


From this point, we begin to correct and fill the cells of the dataset

In [7]:
# Replace the cell with the symbol with its corresponding country
cleaned_df = cleaned_df.with_columns(
    pl.when(pl.col("Country").str.contains("�"))
    .then(pl.lit("Sao Tome and Principe"))
    .otherwise(pl.col("Country"))
    .alias("Country")
)

# Get the correct ISO code abbreviations for each country
isoCodes = PdfReader("../data/raw/CountryCodesISO2.pdf")
text = []
for page in isoCodes.pages:
    page_text = page.extract_text()
    lines = page_text.splitlines()
    # Skip any empty lines and the 'Country Codes ISO2' title
    filtered_lines = [
        line for line in lines if line.strip() and "Country Codes ISO2" not in line
    ]
    text.extend(filtered_lines)

country_code = {}
for line in text:
    parts = line.strip().split(maxsplit=1)
    if len(parts) == 2 and len(parts[0]) == 2 and parts[0].isalpha():
        code, country = parts
        country_code[country.split(" (")[0]] = code.upper()


def get_country_code(country):
    return country_code.get(country)


# Create a new corrected column
cleaned_df = cleaned_df.with_columns(
    pl.when((pl.col("Abbreviation").is_null()) | (pl.col("Abbreviation") == ""))
    .then(pl.col("Country").map_elements(get_country_code, return_dtype=pl.Utf8))
    .otherwise(pl.col("Abbreviation"))
    .alias("Abbreviation")
)

Verify that the changes have been made correctly

In [8]:
symbol_country = (
    cleaned_df.filter(pl.col("Country") == "Sao Tome and Principe")
    .select("Abbreviation")
    .item()
)
symbol_country

'ST'

In [9]:
country_row = cleaned_df.filter(pl.col("Country") == "Republic of Ireland")
country_row

Country,Density(P/Km2),Abbreviation,Agricultural Land(%),Land Area(Km2),Armed Forces size,Birth Rate,Co2-Emissions,CPI,CPI Change (%),Currency-Code,Fertility Rate,Forested Area (%),Gasoline Price,GDP,Gross primary education enrollment (%),Gross tertiary education enrollment (%),Infant mortality,Largest city,Life expectancy,Maternal mortality ratio,Minimum wage,Out of pocket health expenditure,Physicians per thousand,Population,Population: Labor force participation (%),Tax revenue (%),Total tax rate,Unemployment rate,Urban population,Latitude,Longitude
str,i64,str,f64,i64,i64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,i64,f64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64
"""Republic of Ireland""",72,"""IE""",64.5,70273,9000,12.5,37711,106.58,0.9,"""EUR""",1.75,11.0,1.37,3.8870e11,100.9,77.8,3.1,"""Connacht""",82.3,5,10.79,15.2,3.31,5007069,62.1,18.3,26.1,4.93,3133123,53.41291,-8.24389


We're going to update Forest Area (%) by using the information of forest_area_analysis.ipynb:

In [10]:
ls_countries = [
    "Eswatini",
    "North Macedonia",
    "South Sudan",
    "Vatican City",
    "Monaco",
    "Nauru",
    "Palestinian National Authority",
]
ls_values = [29.1, 39.7, 11.3, 0.0, 0.0, 0.0, 1.7]

for i in range(0, len(ls_countries)):
    cleaned_df = cleaned_df.with_columns(
        pl.when(pl.col("Country") == ls_countries[i])
        .then(ls_values[i])
        .otherwise(pl.col("Forested Area (%)"))
        .alias("Forested Area (%)")
    )

cleaned_df[161:163].select(["Country", "Forested Area (%)"])

Country,Forested Area (%)
str,f64
"""South Sudan""",11.3
"""Spain""",36.9


Now, we're going to update Life expectancy column by using Kaggle's Global Life Expectancy dataset: (https://www.kaggle.com/datasets/nafayunnoor/global-life-expectancy-data-1950-2023?select=Preprocessed_Life_Expectancy.csv)

In [11]:
null_values = cleaned_df.filter(
    cleaned_df["Life expectancy"].is_null() | cleaned_df["Life expectancy"].is_nan()
)

null_values.select(["Country", "Life expectancy"])

Country,Life expectancy
str,f64
"""Andorra""",null
"""Eswatini""",null
"""Monaco""",null
"""Nauru""",null
"""North Macedonia""",null
"""Tuvalu""",null


In [12]:
ls_countries = ["Andorra", "Eswatini", "Monaco", "Nauru", "North Macedonia", "Tuvalu"]
ls_values = [84.04, 64.12, 86.37, 63.10, 77.39, 67.10]

for i in range(0, len(ls_countries)):
    cleaned_df = cleaned_df.with_columns(
        pl.when(pl.col("Country") == ls_countries[i])
        .then(ls_values[i])
        .otherwise(pl.col("Life expectancy"))
        .alias("Life expectancy")
    )

Filter countries where latitude or longitude is null or empty

In [13]:
null_values = cleaned_df.filter(
    cleaned_df["Latitude"].is_null()
    | cleaned_df["Latitude"].is_nan()
    | cleaned_df["Longitude"].is_null()
    | cleaned_df["Longitude"].is_nan()
)

null_values.select(["Country", "Latitude", "Longitude"])

Country,Latitude,Longitude
str,f64,f64
"""Sao Tome and Principe""",null,null


To update this column, we're going to check the values using the following website: https://www.latlong.net/place/s-o-tom-and-pr-ncipe-island-25177.html

In [14]:
cleaned_df = cleaned_df.with_columns(
    pl.when(pl.col("Country") == "Sao Tome and Principe")
    .then(0.255436)
    .otherwise(pl.col("Latitude"))
    .alias("Latitude")
)

cleaned_df = cleaned_df.with_columns(
    pl.when(pl.col("Country") == "Sao Tome and Principe")
    .then(6.602781)
    .otherwise(pl.col("Longitude"))
    .alias("Longitude")
)

Let's fill Co2-Emissions column:

In [15]:
null_values = cleaned_df.filter(
    cleaned_df["Co2-Emissions"].is_null() | cleaned_df["Co2-Emissions"].is_nan()
)

null_values.select(["Country", "Co2-Emissions"])

Country,Co2-Emissions
str,i64
"""Eswatini""",null
"""Monaco""",null
"""Nauru""",null
"""North Macedonia""",null
"""San Marino""",null


Now, we are going to display the number of missing values in each column. 

In [16]:
missing_values = cleaned_df.select(
    [
        pl.col(col).is_null().sum().alias(f"{col} missing values")
        for col in cleaned_df.columns
    ]
)
missing_values

Country missing values,Density(P/Km2) missing values,Abbreviation missing values,Agricultural Land(%) missing values,Land Area(Km2) missing values,Armed Forces size missing values,Birth Rate missing values,Co2-Emissions missing values,CPI missing values,CPI Change (%) missing values,Currency-Code missing values,Fertility Rate missing values,Forested Area (%) missing values,Gasoline Price missing values,GDP missing values,Gross primary education enrollment (%) missing values,Gross tertiary education enrollment (%) missing values,Infant mortality missing values,Largest city missing values,Life expectancy missing values,Maternal mortality ratio missing values,Minimum wage missing values,Out of pocket health expenditure missing values,Physicians per thousand missing values,Population missing values,Population: Labor force participation (%) missing values,Tax revenue (%) missing values,Total tax rate missing values,Unemployment rate missing values,Urban population missing values,Latitude missing values,Longitude missing values
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,5,0,22,4,5,18,14,14,5,0,18,0,5,10,4,4,0,12,43,5,5,0,17,24,10,17,3,0,0


Finally, we are going to save the processed dataset in a new file, in order to use it in the next notebooks.

In [17]:
cleaned_df.write_csv("../data/cleaned/data.csv")